# Initial SVC Model

In [8]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import scale
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

In [44]:
df = pd.read_csv("./creditcard.csv")
df.shape

(284807, 31)

In [45]:
df.Class.value_counts().max() / df.Class.value_counts().sum()

0.9982725143693799

In [46]:
df = df.dropna()

In [47]:
X = df.drop('Class', axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
svm = SVC(kernel='linear', random_state=45)
svm.fit(X_train, y_train)

In [14]:
y_pred = svm.predict(X_test)

In [19]:
print("Accuracy (roc_auc_score):", roc_auc_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8927076633973792

Confusion Matrix:
 [[56847    17]
 [   21    77]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.82      0.79      0.80        98

    accuracy                           1.00     56962
   macro avg       0.91      0.89      0.90     56962
weighted avg       1.00      1.00      1.00     56962



# Random Undersampling + Anomaly Detection

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

In [2]:
df = pd.read_csv("./creditcard.csv")

In [3]:
X = df.drop('Class', axis=1)
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
def train_evaluate(undersampling_ratio, X_train, y_train, X_test, y_test):
    # random undersampling
    rus = RandomUnderSampler(sampling_strategy=undersampling_ratio, random_state=42)
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
    
    # Filter out non-fradulent data from the resampled training set
    X_train_normal = X_train_resampled[y_train_resampled == 0]

    # Train a One-Class SVM model on the non-fradulent data
    one_class_svm = OneClassSVM(kernel='rbf', nu=0.01, gamma='scale')
    one_class_svm.fit(X_train_normal)
    y_pred = one_class_svm.predict(X_test)

    # # Convert the predicted labels to match the original labels (0 for normal, 1 for anomaly/fraud)
    y_pred = np.where(y_pred == 1, 0, 1)

    print(f"Undersampling Ratio: {undersampling_ratio * 100}%")
    print("Accuracy (roc_auc_score):", roc_auc_score(y_test, y_pred))
    print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    print("\n")


In [8]:
# Ratio Selection
print(f"Percentage of non-fradulent records: majority class:{(df.Class.value_counts().max() / df.Class.value_counts().sum()).round(3)}")
print(f"Percentage of fraud records (minority class):{(df.Class.value_counts().min() / df.Class.value_counts().sum()).round(3)}")

Percentage of non-fradulent records: majority class:0.998
Percentage of fraud records (minority class):0.002


In [7]:
undersampling_ratios = [0.7, 0.4, 0.1, 0.01, 0.002]

for ratio in undersampling_ratios:
    train_evaluate(ratio, X_train, y_train, X_test, y_test)

Undersampling Ratio: 70.0%
Accuracy (roc_auc_score): 0.8970529756640979

Confusion Matrix:
 [[49798  7066]
 [    8    90]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.88      0.93     56864
           1       0.01      0.92      0.02        98

    accuracy                           0.88     56962
   macro avg       0.51      0.90      0.48     56962
weighted avg       1.00      0.88      0.93     56962



Undersampling Ratio: 40.0%
Accuracy (roc_auc_score): 0.9110073228785043

Confusion Matrix:
 [[51385  5479]
 [    8    90]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.90      0.95     56864
           1       0.02      0.92      0.03        98

    accuracy                           0.90     56962
   macro avg       0.51      0.91      0.49     56962
weighted avg       1.00      0.90      0.95     56962



Undersampling Ratio: 10.0%
Accuracy (roc_auc_s